# 📓 **AICChronicle | Microscopy Database**
___







___
# 🔗 Load AIC Database from GitHub

This notebook can seamlessly integrate with your microscopy’s centralized database, which can be hosted publicly or privately on GitHub. To connect to your GitHub repository URL below. If your repository is private, you'll also need to provide a Personal Access Token.

Check how to generate a token [here](https://github.com/CellMigrationLab/DataManagement/blob/main/docs/others/Generate_GitHub_Token.md).




In [ ]:
# @markdown ### Run to import the necessary libraries

from pathlib import Path
import sys

# Metadata
current_version = "0.0.1"
notebook_name = "Microscopy_Database"

# ---------------------------------------------------------
# 1. Environment Detection & Installation (Colab Only)
# ---------------------------------------------------------
if 'google.colab' in sys.modules:
    print("🚀 Detected Google Colab. Starting installation...")

    from google.colab import userdata, drive

    try:
        token = userdata.get('GITHUB_TOKEN')
    except Exception:
        print("Error: Secret 'GITHUB_TOKEN' not found. Please add it in the sidebar (🔑).")
        token = None

    if token:
        user = 'CellMigrationLab'
        repo = 'AIC-Chronicle'
        !pip install -q "AIC-Chronicle @ git+https://{token}@github.com/{user}/{repo}.git"

else:
    print("done")

from aic_chronicle.notebook_repo_ui import build_connect_repository_panel, build_push_panel

if "state" not in globals():
    state = {"base_path": None}

build_connect_repository_panel(state)
print("✅ Done")


In [ ]:
# @title Microscope Management System v6.0 { display-mode: "form" }

from __future__ import annotations

import re
import hashlib
import traceback
import datetime as dt
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple
import copy

import yaml
import ipywidgets as W
from IPython.display import display, HTML

# ---------------------------- Configuration ----------------------------

def get_base_path() -> Path:
    # 1. Check for the new AIC Database loader (Priority)
    if "AIC_PATH" in globals():
        return Path(AIC_PATH).expanduser().resolve()

    # 2. Check for the legacy LabChronicle loader
    if "state" in globals() and isinstance(state, dict) and state.get("base_path"):
        return Path(state["base_path"]).expanduser().resolve()

    # 3. Fallback to current directory
    return Path.cwd().resolve()

BASE_PATH = get_base_path()

# Determine the correct subfolder based on repo structure
if (BASE_PATH / "instruments").exists():
    # New AIC-Chronicles Structure
    MIC_DIR = BASE_PATH / "instruments"
elif (BASE_PATH / "lab_database" / "light_microscopy").exists():
    # Legacy LabChronicle Structure
    MIC_DIR = BASE_PATH / "lab_database" / "light_microscopy"
else:
    # Default to new structure (will create empty folder if missing)
    MIC_DIR = BASE_PATH / "instruments"

MIC_DIR.mkdir(parents=True, exist_ok=True)

print(f"📂 Loading microscopes from: {MIC_DIR}")

# ---------------------------- Controlled Vocabularies (OME Aligned) ----------------------------

COMMON_MANUFACTURERS = [
    "Other", "Zeiss", "Leica Microsystems", "Nikon", "Olympus/Evident",
    "Abberior", "Andor", "Hamamatsu", "Lumencor", "Excelitas", "Chroma", "Semrock"
]

STAND_ORIENTATION = [
    ("Upright", "upright"),
    ("Inverted", "inverted"),
    ("Stereo/Macro", "stereo"),
    ("Other", "other")
]

SCANNER_TYPES = [
    ("None (Widefield)", "none"),
    ("Point Scanner (Galvo)", "galvo"),
    ("Resonant Scanner", "resonant"),
    ("Spinning Disk", "spinning_disk"),
    ("Polygon", "polygon"),
    ("Acousto-Optic (AOD/AOM)", "acousto_optic"),
    ("Stage Scanning", "stage_scanning"),
    ("Other", "other"),
]

# Modalities - Aligned closer to OME-XML
MODALITY_OPTIONS: List[Tuple[str, str]] = [
    ("Widefield Fluorescence", "widefield_fluorescence"),
    ("Brightfield (Transmitted)", "transmitted_brightfield"),
    ("Phase Contrast", "phase_contrast"),
    ("DIC (Differential Interference Contrast)", "DIC"),
    ("Confocal - Point Scanning (LSM)", "confocal_point"),
    ("Confocal - Spinning Disk (SDC)", "confocal_spinning_disk"),
    ("TIRF (Total Internal Reflection)", "TIRF"),
    ("Light Sheet (SPIM/LSFM)", "light_sheet"),
    ("Multiphoton / Two-Photon", "multiphoton"),
    ("Super-Res: SIM", "SIM"),
    ("Super-Res: STED", "STED"),
    ("Super-Res: SMLM (PALM/STORM/DNA-PAINT)", "SMLM"),
    ("Super-Res: Airyscan/ISM", "ISM"),
    ("Spectral Imaging", "spectral_imaging"),
    ("Other", "other"),
]

MODULE_OPTIONS: List[Tuple[str, str]] = [
    ("Incubation (Temp/CO2)", "incubation"),
    ("Hardware Autofocus (Definite/PFS/Z-drift)", "hardware_autofocus"),
    ("FRAP Module", "FRAP"),
    ("FCS Module", "FCS"),
    ("FLIM Module", "FLIM"),
    ("Optogenetics / Photo-manipulation", "optogenetics"),
    ("Adaptive Optics", "adaptive_optics"),
    ("Other", "other"),
]

LIGHT_SOURCE_KINDS = [
    ("Laser (Diode/Gas/Solid State)", "laser"),
    ("White Light Laser (WLL)", "white_light_laser"),
    ("LED", "led"),
    ("Arc Lamp (Mercury/Xenon)", "arc_lamp"),
    ("Halogen Lamp", "halogen_lamp"),
    ("Metal Halide", "metal_halide"),
    ("Supercontinuum", "supercontinuum"),
    ("Other", "other"),
]

DETECTOR_KINDS = [
    ("PMT (Standard)", "PMT"),
    ("PMT (GaAsP / High Sens)", "GaAsP_PMT"),
    ("Hybrid (HyD/GaAsP)", "HyD"),
    ("sCMOS Camera", "sCMOS"),
    ("EMCCD Camera", "EMCCD"),
    ("CCD Camera", "CCD"),
    ("APD / SPAD", "APD"),
    ("Photodiode (Transmitted)", "photodiode"),
    ("Other", "other"),
]

IMMERSION_OPTIONS: List[Tuple[str, str]] = [
    ("Air (Dry)", "air"),
    ("Oil", "oil"),
    ("Water", "water"),
    ("Glycerol", "glycerol"),
    ("Silicone", "silicone"),
    ("Multi-immersion", "multi"),
    ("Dipping (Direct Water)", "dipping"),
    ("Other", "other"),
]

OBJECTIVE_CORRECTION = [
    ("Plan Apochromat", "plan_apochromat"),
    ("Apochromat", "apochromat"),
    ("Plan Fluorite / Semi-Apo", "plan_fluorite"),
    ("Fluorite", "fluorite"),
    ("Plan Achromat", "plan_achromat"),
    ("Achromat", "achromat"),
    ("Super-Apochromat", "super_apochromat"),
    ("Other", "other")
]

# ---------------------------- Robust dropdown helpers ----------------------------

def _options_values(options: Any) -> List[Any]:
    if isinstance(options, dict):
        return list(options.values())
    vals: List[Any] = []
    for o in list(options or []):
        if isinstance(o, (tuple, list)) and len(o) == 2:
            vals.append(o[1])
        else:
            vals.append(o)
    return vals

def _normalize_token(x: str) -> str:
    return str(x).strip().lower().replace(" ", "_").replace("-", "_")

def _coerce_to_allowed(value: Any, allowed: List[Any], fallback: Any) -> Any:
    if not allowed:
        return None
    if value in allowed:
        return value

    # Handle tuple input
    if isinstance(value, (tuple, list)) and len(value) == 2:
        if value[1] in allowed: return value[1]
        if value[0] in allowed: return value[0]

    # Handle string normalization
    if isinstance(value, str):
        v = value.strip()
        if v in allowed: return v
        vn = _normalize_token(v)
        for a in allowed:
            if isinstance(a, str) and _normalize_token(a) == vn:
                return a
    return fallback

def make_dropdown(options: Any, value: Any = None, **kwargs) -> W.Dropdown:
    dd = W.Dropdown(options=[], **kwargs)
    dd.options = options
    allowed = _options_values(dd.options)
    fallback = allowed[0] if allowed else None
    safe_value = _coerce_to_allowed(value, allowed, fallback)
    if safe_value is not None:
        dd.value = safe_value
    return dd

def make_combobox(options: List[str], value: str = "", placeholder: str = "", **kwargs) -> W.Combobox:
    """Creates a text field with autocomplete suggestions."""
    return W.Combobox(
        options=options,
        value=str(value) if value else "",
        placeholder=placeholder,
        ensure_option=False,
        **kwargs
    )

# ---------------------------- UI Styling & Helpers ----------------------------

class UIHelper:
    _css_injected = False

    @staticmethod
    def inject_css():
        if UIHelper._css_injected: return
        UIHelper._css_injected = True
        display(HTML("""
        <style>
            :root { --primary: #2563eb; --bg-gray: #f8fafc; --border: #e2e8f0; --text: #334155; }
            .mic-container { font-family: 'Segoe UI', system-ui, sans-serif; color: var(--text); max-width: 950px; }

            .mic-header {
                font-size: 13px; font-weight: 700; color: #0f172a;
                text-transform: uppercase; letter-spacing: 0.05em;
                border-bottom: 2px solid var(--border); padding-bottom: 8px; margin: 25px 0 15px 0;
            }

            .mic-sublabel { font-size: 11px; font-weight: 600; color: #64748b; margin-bottom: 2px; }

            .mic-card {
                background: white; border: 1px solid var(--border); border-radius: 8px;
                padding: 15px; margin-bottom: 10px;
                box-shadow: 0 1px 2px rgba(0,0,0,0.03);
            }

            .mic-list-row {
                background: #fff; border: 1px solid #cbd5e1; border-left: 4px solid var(--primary);
                border-radius: 4px; padding: 10px; margin-bottom: 8px;
            }

            .mic-toolbar {
                background: var(--bg-gray); border: 1px solid var(--border);
                padding: 12px; border-radius: 8px; margin-bottom: 15px;
            }

            .widget-label { font-weight: bold; }
        </style>
        """))

    @staticmethod
    def header(text: str) -> W.HTML:
        return W.HTML(f"<div class='mic-header'>{text}</div>")

    @staticmethod
    def field(label: str, widget: W.Widget, width: str = "auto") -> W.VBox:
        lbl = W.HTML(f"<div class='mic-sublabel'>{label}</div>")
        widget.layout.width = "100%"
        return W.VBox([lbl, widget], layout=W.Layout(width=width, margin='0 10px 5px 0'))

# ---------------------------- Component Widgets ----------------------------

class RepeatList(W.VBox):
    """A list manager that handles Add/Remove logic for complex rows."""
    def __init__(self, title: str, row_factory, row_serializer, add_label: str = "Add Item"):
        self.row_factory = row_factory
        self.row_serializer = row_serializer

        self.header = UIHelper.header(title)
        self.items_box = W.VBox([])
        self.btn_add = W.Button(description=f" {add_label}", icon="plus", button_style='', layout=W.Layout(width="auto"))
        self.btn_add.on_click(lambda _: self.add_row())

        super().__init__([self.header, self.items_box, self.btn_add])
        self.add_class("mic-card")

    def add_row(self, data=None):
        content_widget = self.row_factory(data)

        # Delete button container
        btn_del = W.Button(icon="trash", button_style="danger", layout=W.Layout(width="36px", height="36px"))

        # Row Container
        row = W.HBox([content_widget, btn_del], layout=W.Layout(width="100%", align_items="center"))
        row.add_class("mic-list-row")

        btn_del.on_click(lambda _: self._remove(row))

        # Add to list
        self.items_box.children = tuple(list(self.items_box.children) + [row])

    def _remove(self, row_box):
        kids = list(self.items_box.children)
        if row_box in kids:
            kids.remove(row_box)
            self.items_box.children = tuple(kids)

    def get_data(self) -> List[dict]:
        out = []
        for row in self.items_box.children:
            # The content widget is the first child of the HBox row
            content = row.children[0]
            try:
                val = self.row_serializer(content)
                if val: out.append(val)
            except Exception:
                pass
        return out

    def load_data(self, data_list: List[dict]):
        self.items_box.children = ()
        for item in (data_list or []):
            self.add_row(item)

class SimpleChoiceList(RepeatList):
    """Specialized RepeatList for simple Dropdowns (Modalities/Modules)."""
    def __init__(self, title: str, options: List[Tuple[str, str]], add_label: str):
        self.options = options
        super().__init__(title, self._factory, self._serializer, add_label)

    def _factory(self, value=None):
        return make_dropdown(self.options, value=value) # Width handled by parent

    def _serializer(self, widget):
        return widget.value

    def get_data(self) -> List[str]:
        # Deduped list of strings
        raw = super().get_data()
        return list(set(raw))

# ---------------------------- Tabs ----------------------------

class GeneralTab:
    def __init__(self):
        self.name = W.Text(placeholder="e.g. Zeiss LSM 980 Airyscan")
        self.id_field = W.Text(disabled=True, placeholder="Auto-generated on save")

        # Use Combobox for Manufacturer to allow "Other" custom entry while suggesting common ones
        self.manuf = make_combobox(COMMON_MANUFACTURERS, placeholder="Select or type manufacturer")
        self.model = W.Text(placeholder="e.g. AxioObserver 7")
        self.orient = make_dropdown(STAND_ORIENTATION, value="other")
        self.notes = W.Textarea(placeholder="Room number, custodian, or general status...", layout=W.Layout(height="80px"))

        self.layout = W.VBox([
            UIHelper.header("Instrument Identity"),
            W.HBox([
                UIHelper.field("Display Name (Unique)", self.name, width="60%"),
                UIHelper.field("System ID", self.id_field, width="35%")
            ]),
            W.HBox([
                UIHelper.field("Manufacturer", self.manuf, width="48%"),
                UIHelper.field("Model Base", self.model, width="48%")
            ]),
            UIHelper.field("Stand Orientation", self.orient, width="50%"),
            UIHelper.field("General Notes", self.notes, width="100%")
        ])
        self.layout.add_class("mic-card")

    def load(self, data: dict):
        inst = data.get("instrument", {}) or {}
        self.name.value = str(inst.get("display_name", ""))
        self.id_field.value = str(inst.get("instrument_id", ""))
        self.manuf.value = str(inst.get("manufacturer", ""))
        self.model.value = str(inst.get("model", ""))
        self.orient.value = _coerce_to_allowed(inst.get("stand_orientation", "other"), STAND_ORIENTATION, "other")
        self.notes.value = str(inst.get("notes", ""))

    def get(self) -> dict:
        return {
            "display_name": self.name.value,
            "instrument_id": self.id_field.value,
            "manufacturer": self.manuf.value,
            "model": self.model.value,
            "stand_orientation": self.orient.value,
            "notes": self.notes.value
        }

class ConfigTab:
    def __init__(self):
        self.modalities = SimpleChoiceList("Imaging Modalities", MODALITY_OPTIONS, "Add Modality")
        self.modules = SimpleChoiceList("Hardware Modules", MODULE_OPTIONS, "Add Module")

        self.acq_name = make_combobox(["ZEN", "LAS X", "NIS-Elements", "CellSens", "MicroManager", "SlideBook"], placeholder="Software Name")
        self.acq_ver = W.Text(placeholder="e.g. 3.4.1")

        self.sw_box = W.VBox([
            UIHelper.header("Software"),
            W.HBox([
                UIHelper.field("Acquisition Software", self.acq_name, width="50%"),
                UIHelper.field("Version", self.acq_ver, width="40%")
            ])
        ])
        self.sw_box.add_class("mic-card")

        self.layout = W.HBox([
            W.VBox([self.modalities, self.modules], layout=W.Layout(flex="1", margin="0 10px 0 0")),
            W.VBox([self.sw_box], layout=W.Layout(flex="1"))
        ])

    def load(self, data: dict):
        self.modalities.load_data(data.get("modalities", []))
        self.modules.load_data(data.get("modules", []))
        soft = (data.get("software", {}) or {}).get("acquisition", {}) or {}
        self.acq_name.value = str(soft.get("name", ""))
        self.acq_ver.value = str(soft.get("version", ""))

    def get(self) -> Tuple[List[str], List[str], dict]:
        return (
            self.modalities.get_data(),
            self.modules.get_data(),
            {"name": self.acq_name.value, "version": self.acq_ver.value}
        )

class HardwareTab:
    def __init__(self):
        self.scan_type = make_dropdown(SCANNER_TYPES, value="none")
        self.scan_notes = W.Text(placeholder="e.g. Resonant 8kHz")

        self.scanner_box = W.VBox([
            UIHelper.header("Scanner System"),
            W.HBox([
                UIHelper.field("Scanner Type", self.scan_type, width="40%"),
                UIHelper.field("Details", self.scan_notes, width="55%")
            ])
        ])
        self.scanner_box.add_class("mic-card")

        self.lights = RepeatList("Light Sources (Excitation)", self._make_light, self._read_light, "Add Light Source")
        self.detectors = RepeatList("Detectors (Cameras/PMTs)", self._make_det, self._read_det, "Add Detector")

        self.layout = W.VBox([self.scanner_box, self.lights, self.detectors])

    # --- Light Source Logic ---
    def _make_light(self, d=None):
        d = d or {}
        # Row 1
        kind = make_dropdown(LIGHT_SOURCE_KINDS, value=d.get("kind", "laser"))
        man = make_combobox(COMMON_MANUFACTURERS, value=d.get("manufacturer", ""), placeholder="Manufacturer")
        # Row 2
        mod = W.Text(value=str(d.get("model", "")), placeholder="Model / Name")
        wl = W.IntText(value=int(d.get("wavelength_nm") or 0))
        power = W.Text(value=str(d.get("power", "")), placeholder="e.g. 50mW")

        r1 = W.HBox([UIHelper.field("Type", kind, "45%"), UIHelper.field("Manufacturer", man, "50%")])
        r2 = W.HBox([
            UIHelper.field("Model", mod, "35%"),
            UIHelper.field("Wavelength (nm)", wl, "25%"),
            UIHelper.field("Power/Intensity", power, "30%")
        ])
        return W.VBox([r1, r2], layout=W.Layout(width="100%"))

    def _read_light(self, w):
        # Unpack VBox structure
        r1, r2 = w.children
        wl_val = r2.children[1].children[1].value # Row 2, Col 2, Widget
        return {
            "kind": r1.children[0].children[1].value,
            "manufacturer": r1.children[1].children[1].value,
            "model": r2.children[0].children[1].value,
            "wavelength_nm": int(wl_val) if wl_val else None,
            "power": r2.children[2].children[1].value
        }

    # --- Detector Logic ---
    def _make_det(self, d=None):
        d = d or {}
        kind = make_dropdown(DETECTOR_KINDS, value=d.get("kind", "PMT"))
        man = make_combobox(COMMON_MANUFACTURERS, value=d.get("manufacturer", ""), placeholder="Manufacturer")
        mod = W.Text(value=str(d.get("model", "")), placeholder="e.g. Flash 4.0")

        return W.HBox([
            UIHelper.field("Type", kind, "30%"),
            UIHelper.field("Manufacturer", man, "30%"),
            UIHelper.field("Model", mod, "35%")
        ], layout=W.Layout(width="100%"))

    def _read_det(self, w):
        return {
            "kind": w.children[0].children[1].value,
            "manufacturer": w.children[1].children[1].value,
            "model": w.children[2].children[1].value,
        }

    def load(self, data: dict):
        hw = data.get("hardware", {}) or {}
        sc = hw.get("scanner", {}) or {}
        self.scan_type.value = _coerce_to_allowed(sc.get("type", "none"), SCANNER_TYPES, "none")
        self.scan_notes.value = str(sc.get("notes", ""))
        self.lights.load_data(hw.get("light_sources", []) or [])
        self.detectors.load_data(hw.get("detectors", []) or [])

    def get(self) -> dict:
        return {
            "scanner": {"type": self.scan_type.value, "notes": self.scan_notes.value},
            "light_sources": self.lights.get_data(),
            "detectors": self.detectors.get_data()
        }

class ObjectivesTab:
    def __init__(self):
        self.objs = RepeatList("Objectives", self._make_obj, self._read_obj, "Add Objective")
        self.layout = W.VBox([self.objs])

    def _make_obj(self, d=None):
        d = d or {}
        # Line 1: Basic Specs
        mag = W.FloatText(value=float(d.get("magnification") or 0))
        na = W.FloatText(value=float(d.get("numerical_aperture") or 0))
        imm = make_dropdown(IMMERSION_OPTIONS, value=d.get("immersion", "air"))
        corr = make_dropdown(OBJECTIVE_CORRECTION, value=d.get("correction", "plan_apochromat"))

        # Line 2: Identity
        man = make_combobox(COMMON_MANUFACTURERS, value=d.get("manufacturer", ""), placeholder="Make")
        mod = W.Text(value=str(d.get("model", "")), placeholder="e.g. Plan-Apo 63x/1.4 Oil")
        wd = W.Text(value=str(d.get("working_distance", "")), placeholder="e.g. 0.17mm")

        row1 = W.HBox([
            UIHelper.field("Mag (x)", mag, "15%"),
            UIHelper.field("NA", na, "15%"),
            UIHelper.field("Immersion", imm, "30%"),
            UIHelper.field("Correction", corr, "30%"),
        ])
        row2 = W.HBox([
            UIHelper.field("Manufacturer", man, "30%"),
            UIHelper.field("Full Name/Model", mod, "40%"),
            UIHelper.field("Working Dist.", wd, "20%")
        ])
        return W.VBox([row1, row2], layout=W.Layout(width="100%"))

    def _read_obj(self, w):
        r1, r2 = w.children
        mag = r1.children[0].children[1].value
        na = r1.children[1].children[1].value

        if mag == 0: return None # Filter empty

        return {
            "magnification": mag,
            "numerical_aperture": na,
            "immersion": r1.children[2].children[1].value,
            "correction": r1.children[3].children[1].value,
            "manufacturer": r2.children[0].children[1].value,
            "model": r2.children[1].children[1].value,
            "working_distance": r2.children[2].children[1].value,
        }

    def load(self, data: dict):
        hw = data.get("hardware", {}) or {}
        self.objs.load_data(hw.get("objectives", []) or [])

    def get(self) -> list:
        return self.objs.get_data()

# ---------------------------- Engine ----------------------------

class MicroscopeEngine:
    def __init__(self, root_dir: Path):
        self.root = root_dir

    def list_microscopes(self) -> List[Tuple[str, str]]:
        files = sorted(list(self.root.glob("*.yml")) + list(self.root.glob("*.yaml")))
        return [(f.stem, f.stem) for f in files]

    def _get_empty_schema(self) -> dict:
        return {
            "instrument": { "display_name": "", "instrument_id": "", "manufacturer": "", "model": "", "stand_orientation": "other", "notes": "" },
            "modalities": [],
            "software": {"acquisition": {"name": "", "version": ""}},
            "hardware": { "scanner": {"type": "none"}, "light_sources": [], "detectors": [], "objectives": [] },
            "modules": []
        }

    def load_microscope(self, name: str) -> dict:
        if not name: return self._get_empty_schema()
        fpath = self.root / f"{name}.yaml"
        if not fpath.exists(): fpath = self.root / f"{name}.yml"
        if not fpath.exists(): return self._get_empty_schema()

        try:
            with fpath.open("r", encoding="utf-8") as f:
                data = yaml.safe_load(f) or {}
        except Exception:
            return self._get_empty_schema()

        default = self._get_empty_schema()
        self._safe_merge(default, data)
        return default

    def _safe_merge(self, target, source):
        for k, v in (source or {}).items():
            if k in target and isinstance(target[k], dict) and isinstance(v, dict):
                self._safe_merge(target[k], v)
            else:
                target[k] = v

    def save_microscope(self, current_name: Optional[str], data: dict) -> str:
        display_name = str(data.get("instrument", {}).get("display_name", "")).strip()
        if not display_name: raise ValueError("Display Name is required.")

        canon = re.sub(r"\s+", " ", display_name).strip().lower()
        hid = hashlib.sha1(canon.encode("utf-8")).hexdigest()[:8]
        data["instrument"]["instrument_id"] = f"scope-{hid}"

        safe_name = re.sub(r"[\/\\\:\*\?\"\<\>\|]+", "-", display_name).strip().strip(".")
        if not safe_name: safe_name = "Microscope"

        target_path = self.root / f"{safe_name}.yaml"
        if current_name and current_name != safe_name:
            for ext in (".yaml", ".yml"):
                (self.root / f"{current_name}{ext}").unlink(missing_ok=True)

        with target_path.open("w", encoding="utf-8") as f:
            yaml.safe_dump(data, f, sort_keys=False, default_flow_style=False, width=100)
        return safe_name

# ---------------------------- Main App ----------------------------

class MicroscopeApp:
    def __init__(self):
        UIHelper.inject_css()
        self.engine = MicroscopeEngine(MIC_DIR)

        # Toolbar
        self.dd_select = make_dropdown([("— Select Scope —", "")], layout=W.Layout(width="300px"))
        self.btn_refresh = W.Button(icon="refresh", layout=W.Layout(width="40px"), tooltip="Refresh List")
        self.btn_new = W.Button(description="New", icon="plus", button_style="info", layout=W.Layout(width="100px"))
        self.btn_save = W.Button(description="Save", icon="save", button_style="success", layout=W.Layout(width="100px"))
        self.status = W.HTML("System Ready.", layout=W.Layout(margin="0 0 0 15px", padding="5px"))

        self.toolbar = W.HBox([self.btn_refresh, self.dd_select, self.btn_new, self.btn_save, self.status])
        self.toolbar.add_class("mic-toolbar")

        # Tabs
        self.tab_gen = GeneralTab()
        self.tab_cfg = ConfigTab()
        self.tab_hw = HardwareTab()
        self.tab_obj = ObjectivesTab()

        self.tabs = W.Tab(children=[
            self.tab_gen.layout,
            self.tab_cfg.layout,
            self.tab_hw.layout,
            self.tab_obj.layout
        ])
        for i, t in enumerate(["General Info", "Config & SW", "Hardware & Lights", "Objectives"]):
            self.tabs.set_title(i, t)

        self.container = W.VBox([self.toolbar, self.tabs])
        self.container.add_class("mic-container")

        # Events
        self.btn_refresh.on_click(self.refresh_list)
        self.dd_select.observe(self.on_select, names="value")
        self.btn_new.on_click(self.on_new)
        self.btn_save.on_click(self.on_save)

        self.current_filename: Optional[str] = None
        self.current_data: dict = self.engine._get_empty_schema()

        self.refresh_list()
        self.on_new(None)

    def log(self, msg: str, color="#334155"):
        ts = dt.datetime.now().strftime("%H:%M:%S")
        self.status.value = f"<span style='color:{color}; font-weight:500;'>[{ts}] {msg}</span>"

    def refresh_list(self, _=None):
        files = self.engine.list_microscopes()
        cur = self.dd_select.value
        opts = [("— Select Scope —", "")] + [(stem, stem) for stem, _ in files]
        self.dd_select.options = opts

        # Restore selection if valid
        allowed = _options_values(self.dd_select.options)
        if cur in allowed: self.dd_select.value = cur
        else: self.dd_select.value = ""

    def on_select(self, change):
        val = change["new"]
        if not val: return
        self.current_filename = val
        try:
            data = self.engine.load_microscope(val)
            self.current_data = copy.deepcopy(data)
            self._distribute_data(data)
            self.log(f"Loaded: {val}", "#166534") # Green
        except Exception as e:
            self.log(f"Error: {e}", "#991b1b") # Red
            traceback.print_exc()

    def on_new(self, _):
        self.current_filename = None
        self.dd_select.value = ""
        empty = self.engine._get_empty_schema()
        self.current_data = copy.deepcopy(empty)
        self._distribute_data(empty)
        self.log("New workspace created.", "#2563eb") # Blue

    def _distribute_data(self, data: dict):
        self.tab_gen.load(data)
        self.tab_cfg.load(data)
        self.tab_hw.load(data)
        self.tab_obj.load(data)

    def on_save(self, _):
        try:
            final_data = copy.deepcopy(self.current_data)
            final_data["instrument"] = self.tab_gen.get()

            mods, modules, soft = self.tab_cfg.get()
            final_data["modalities"] = mods
            final_data["modules"] = modules
            final_data["software"] = {"acquisition": soft}

            hw = self.tab_hw.get()
            final_data["hardware"] = hw
            final_data["hardware"]["objectives"] = self.tab_obj.get()

            new_name = self.engine.save_microscope(self.current_filename, final_data)
            self.current_filename = new_name
            self.refresh_list()
            self.dd_select.value = new_name
            self.log(f"Successfully saved: {new_name}.yaml", "#166534")

        except Exception as e:
            self.log(f"Save Failed: {e}", "#991b1b")
            traceback.print_exc()

# ---------------------------- Launch ----------------------------

app = MicroscopeApp()
display(app.container)

In [ ]:
# @title Run to push the edits to GitHub

# 1. Define which folders to watch for changes
folders_to_sync = ["instruments"]

# 2. Build the panel
push_panel = build_push_panel(
    folder_list=folders_to_sync,
    default_commit_message="Update microscope configurations",
    pr_title="Update Instruments Database",
    pr_body="This PR updates the instrument YAML.",
    state=state
)

display(push_panel)
